In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.new_orleans_harbor_pd_pprr import clean_personnel
from lib.columns import rearrange_personnel_columns, rearrange_personnel_history_columns

In [2]:
df = clean_personnel()
df

,first_name,last_name,rank_desc,hourly_salary,middle_initial,term_year,term_month,term_day,hire_year,hire_month,hire_day,pay_effective_year,pay_effective_month,pay_effective_day,uid,agency,data_production_year
0,casey,adams,police sergeant--port,17.00,m,2010,2,28,1980,1,31,1997,4,14,83d0f42cf88270c9180153b13339b727,New Orleans Harbor PD,2020
1,casey,adams,police sergeant--port,17.68,m,2010,2,28,1980,1,31,2003,12,10,83d0f42cf88270c9180153b13339b727,New Orleans Harbor PD,2020
2,casey,adams,police sergeant--port,18.39,m,2010,2,28,1980,1,31,2004,8,10,83d0f42cf88270c9180153b13339b727,New Orleans Harbor PD,2020
3,casey,adams,police sergeant--port,19.13,m,2010,2,28,1980,1,31,2005,8,10,83d0f42cf88270c9180153b13339b727,New Orleans Harbor PD,2020
4,casey,adams,police sergeant-a,19.13,m,2010,2,28,1980,1,31,2005,8,29,83d0f42cf88270c9180153b13339b727,New Orleans Harbor PD,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,brianne,waters,police officer 3-a,21.60,m,2020,5,11,2001,3,5,2015,10,1,03bdf56f995a6f875d5aa9be240c0ce9,New Orleans Harbor PD,2020
911,brianne,waters,police sergeant-a,24.34,m,2020,5,11,2001,3,5,2018,5,14,03bdf56f995a6f875d5aa9be240c0ce9,New Orleans Harbor PD,2020
912,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,03bdf56f995a6f875d5aa9be240c0ce9,New Orleans Harbor PD,2020
913,brianne,waters,police sergeant-a,25.07,m,2020,5,11,2001,3,5,2018,7,15,03bdf56f995a6f875d5aa9be240c0ce9,New Orleans Harbor PD,2020


In [8]:
rearrange_personnel_columns(df)

,uid,last_name,middle_initial,first_name
0,ac49c8119f,adams,m,casey
1,4500bb1616,additon,p,bernard
2,3d75f7a67d,allsbrook,l,michael
3,b81578b705,anderson jr,NaN,melvin
4,25cd1eb87e,arena,p,bryan
...,...,...,...,...
113,27d88f2164,walker,NaN,winston
114,ec61f23394,walton,m,byron
115,2306549e7a,washington,m,ardiel
116,753604c68d,washington,w,ryan


In [9]:
hist = rearrange_personnel_history_columns(df)
hist.rank_desc.unique()

array(['police sergeant--port', 'police sergeant-a', 'police officer 1-a',
       'police officer 2--port', 'police officer 2-a',
       'police officer 3-a', 'police officer 1--port',
       'police corporal--port', 'police lieutenant-a',
       'police lieutenant--port', 'policy planner 4',
       'police chief--port', 'police chief-a', 'admin specialist 2',
       'admin program specialist--a', 'police captain-a',
       'police captain--port'], dtype=object)